# Project 3 - Article Parsing Toolset

## Team 3 Members

- Matthew Dunbar
- Jeffrei Cher
- Basil James

## Project Problem

To parse article content and extract various data from the article contents

- _**Article Classification into a set of pre-defined categories**_
  (a prediction model)

For Sports Articles:
- _**to identify the sport, and extract game/match scores if present**_
- _(to provide stats summaries if present?)_

## Learning Goal

Develop experience:

- Building and deploying models to leverage in real world applications
- Leveraging custom training of LLMs to provide article analysis.
- Using a tool-based (extensible toolbox) approach to provide multiple analytics features


## Solution

### Approach

LLM-based tool(s) with tool-specific training, with tool-specific engineered prompt(s).
Modular. Function-based.  Extensible.


#### Input: content of the article to analyse

(ideally, the contents directly.  alternate consideration might be to provide a URL, but that would require additional python supporting fucntions to pull, then clean up the contents prior to submission)

#### Output: variable per tool/function

### Tools

#### Article Topic Classifier



#### Sport and Score Extractor


#### Sport Statistics Summarizer


### API Deployment (Input/Output)

Webapp endpoint

## Dataset

- https://www.kaggle.com/datasets/fabiochiusano/medium-articles  
[size: 190k+, categories: multiple tags per article, large set, Includes titles, full articles, and URLs]


#### Retrieve dataset and load into Big Query

## Reference Labs

- Gemini Function Calling
- AutoML for Text Classification - Vertex
- KFP Walkthrogh - Vertex Containerization - Training and Deployment Pipelines